In [47]:
import pandas as pd
import numpy as np

test_pairs = pd.read_parquet("data/test_pairs_ids.parquet")
test_pairs = test_pairs[["user_id","item_id","next_item_id","session_id","timestamp"]].dropna()
test_pairs["user_id"] = test_pairs["user_id"].astype(str)
test_pairs["item_id"] = test_pairs["item_id"].astype(str)
test_pairs["next_item_id"] = test_pairs["next_item_id"].astype(str)

print(test_pairs.shape)
test_pairs.head()


(3163359, 5)


,user_id,item_id,next_item_id,session_id,timestamp
0,user_000001,f4fb4539-90b5-401e-8520-1a0dd4e41a32,df8c41f0-1c02-4060-81ca-035ef3085663,1004,2009-01-22 01:10:26+00:00
1,user_000001,df8c41f0-1c02-4060-81ca-035ef3085663,3c224468-a9c4-43d2-9b26-5973718a4f96,1004,2009-01-22 01:14:22+00:00
2,user_000001,3c224468-a9c4-43d2-9b26-5973718a4f96,8c88d32a-8bb1-4e3c-b30f-73107509c8c6,1004,2009-01-22 01:19:34+00:00
3,user_000001,8c88d32a-8bb1-4e3c-b30f-73107509c8c6,8efa69bc-8453-42c8-8949-2a0afd4c8c99,1004,2009-01-22 01:24:26+00:00
4,user_000001,8efa69bc-8453-42c8-8949-2a0afd4c8c99,07ae32f3-f0aa-4479-bb2d-bc6737a4df44,1004,2009-01-22 01:28:19+00:00


In [48]:
topk = pd.read_parquet("data/topk_markov_ids.parquet")
topk.columns = [c.strip() for c in topk.columns]
print("cols topk:", topk.columns.tolist())

# Formato A: item_id | topk(list)
if "topk" in topk.columns:
    markov_topk = dict(zip(topk["item_id"].astype(str), topk["topk"]))

# Formato B: item_id | next_item_id | rank
else:
    # ajuste os nomes se necessário
    if "next_item_id" not in topk.columns and "next_item" in topk.columns:
        topk = topk.rename(columns={"next_item":"next_item_id"})
    topk["item_id"] = topk["item_id"].astype(str)
    topk["next_item_id"] = topk["next_item_id"].astype(str)
    topk = topk.sort_values(["item_id","rank"])
    markov_topk = topk.groupby("item_id")["next_item_id"].apply(list).to_dict()

print("itens com top-k:", len(markov_topk))


cols topk: ['item_id', 'next_item_id', 'rank', 'count']
itens com top-k: 836712


In [49]:
meta = pd.read_parquet("data/item_meta.parquet")
meta.columns = [c.strip() for c in meta.columns]
# esperado: item_id, artist_name, track_name
meta["item_id"] = meta["item_id"].astype(str)

id_to_name = dict(
    zip(meta["item_id"], meta["artist_name"].fillna("") + " — " + meta["track_name"].fillna(""))
)

def pretty(item_id: str) -> str:
    return id_to_name.get(item_id, item_id)


In [50]:
import numpy as np

als_art = np.load("artifacts/als_pref_model_top200_markov.npz", allow_pickle=True)

user_factors = als_art["user_factors"]
item_factors = als_art["item_factors"]

user_index_to_id = als_art["user_index_to_id"].astype(str)
item_index_to_id = als_art["item_index_to_id"].astype(str)

user_id_to_index = {uid: i for i, uid in enumerate(user_index_to_id)}
item_id_to_index = {iid: i for i, iid in enumerate(item_index_to_id)}

print("user_factors:", user_factors.shape)
print("item_factors:", item_factors.shape)
print("users:", len(user_id_to_index), "| items:", len(item_id_to_index))


user_factors: (990, 64)
item_factors: (852555, 64)
users: 990 | items: 852555


In [51]:
def als_score(user_id: str, item_id: str) -> float:
    uidx = user_id_to_index.get(user_id)
    iidx = item_id_to_index.get(item_id)
    if uidx is None or iidx is None:
        return float("NaN")
    return float(user_factors[uidx] @ item_factors[iidx])


In [52]:
df_context = pd.read_parquet("data/pairs_full.parquet")

# padroniza nomes pro get_recent_context (que espera "item")
if "item_id" in df_context.columns and "item" not in df_context.columns:
    df_context = df_context.rename(columns={"item_id": "item"})

# garantir tipos
df_context["user_id"] = df_context["user_id"].astype(str)
df_context["session_id"] = df_context["session_id"].astype(str)
df_context["item"] = df_context["item"].astype(str)

df_context["timestamp"] = pd.to_datetime(df_context["timestamp"], utc=True, errors="coerce")

print(df_context.columns.tolist())
df_context.head()


['user_id', 'session_id', 'timestamp', 'item', 'next_item', 'item_id', 'next_item_id']


,user_id,session_id,timestamp,item,next_item,item_id,next_item_id
0,user_000001,1,2006-08-13 13:59:20+00:00,Plaid & Bob Jaroc — The Launching Of Big Face,Plaid & Bob Jaroc — Zn Zero,c4633ab1-e715-477f-8685-afa5f2058e42,bc2765af-208c-44c5-b3b0-cf597a646660
1,user_000001,1,2006-08-13 14:03:29+00:00,Plaid & Bob Jaroc — Zn Zero,Plaid & Bob Jaroc — The Return Of Super Barrio...,bc2765af-208c-44c5-b3b0-cf597a646660,aa9c5a80-5cbe-42aa-a966-eb3cfa37d832
2,user_000001,1,2006-08-13 14:10:43+00:00,Plaid & Bob Jaroc — The Return Of Super Barrio...,Tommy Guerrero — Mission Flats,aa9c5a80-5cbe-42aa-a966-eb3cfa37d832,d9b1c1da-7e47-4f97-a135-77260f2f559d
3,user_000001,1,2006-08-13 14:17:40+00:00,Tommy Guerrero — Mission Flats,Artful Dodger — What You Gonna Do?,d9b1c1da-7e47-4f97-a135-77260f2f559d,120bb01c-03e4-465f-94a0-dce5e9fac711
4,user_000001,1,2006-08-13 14:19:06+00:00,Artful Dodger — What You Gonna Do?,Rei Harakami — Joy,120bb01c-03e4-465f-94a0-dce5e9fac711,777ac51f-8ffc-4c44-92b6-a2c75cbc6915


In [53]:
import pandas as pd
import numpy as np

# ========= carregar transition_counts pra mostrar "count=XX" =========
# Ajuste o arquivo conforme o que você tiver (ids vs legível)
# transition_counts.parquet normalmente tem: item_id, next_item_id, count
try:
    trans = pd.read_parquet("data/transition_counts_ids.parquet")
    trans["item_id"] = trans["item_id"].astype(str)
    trans["next_item_id"] = trans["next_item_id"].astype(str)
    trans_counts = {(a,b): int(c) for a,b,c in trans[["item_id","next_item_id","count"]].itertuples(index=False)}
except Exception:
    trans_counts = {}

def get_transition_count(item_id: str, next_item_id: str) -> int | None:
    return trans_counts.get((item_id, next_item_id))

# ========= contexto: últimas N músicas da mesma sessão =========
def get_recent_context(df_events, user_id: str, session_id: str, ts, n=5):
    dfu = df_events[(df_events["user_id"].astype(str) == str(user_id)) &
                    (df_events["session_id"].astype(str) == str(session_id))].copy()
    # garantir timestamp datetime
    dfu["timestamp"] = pd.to_datetime(dfu["timestamp"], utc=True, errors="coerce")
    ts = pd.to_datetime(ts, utc=True, errors="coerce")

    # pega apenas eventos ANTERIORES ao timestamp atual (exclui a música atual)
    dfu_prev = dfu[dfu["timestamp"] < ts].sort_values("timestamp")

    # pega as últimas n anteriores (ou menos, se não existir)
    ctx = dfu_prev["item"].astype(str).tail(n).tolist()
    return ctx

# ========= ranking global (se você tiver) =========
# global_rank_ids_top200.parquet tem: item_id, pop_count, rank
try:
    global_rank = pd.read_parquet("data/global_rank_ids_top200.parquet")
    global_rank["item_id"] = global_rank["item_id"].astype(str)
    rank_map = dict(zip(global_rank["item_id"], global_rank.get("rank", pd.Series([None]*len(global_rank)))))
except Exception:
    rank_map = {}

def get_global_rank(item_id: str):
    return rank_map.get(str(item_id))

# ========= rerank híbrido + relatório =========
def hybrid_recommend_with_report(
    user_id: str,
    current_item_id: str,
    session_id: str | None = None,
    timestamp=None,
    K: int = 20,
    topn_show: int = 10,
    context_n: int = 5,
    df_events=None,
    true_next_item_id: str | None = None
):
    current_item_id = str(current_item_id)
    user_id = str(user_id)

    candidates = markov_topk.get(current_item_id, [])
    candidates = [str(x) for x in candidates][:K]
    candidates = [cid for cid in candidates if cid != str(current_item_id)]
    if len(candidates) == 0:
        print("Sem candidatos do Markov para este item.")
        return None

    # pontuar candidatos pelo ALS
    scored = [(cid, als_score(user_id, cid)) for cid in candidates]
    best_id, best_score = max(scored, key=lambda x: x[1])

    # count Markov do par (Y->X)
    c = get_transition_count(current_item_id, best_id)

    # explicação no seu formato
    exp = (
        f"Você está ouvindo {pretty(best_id)} porque é a que tem mais afinidade com o seu gosto musical "
        f"entre as músicas mais escutadas depois de {pretty(current_item_id)}."
    )

    # ----- imprimir relatório -----
    print("Música atual:   ", pretty(current_item_id))
    if true_next_item_id is not None:
        print("Próxima real:   ", pretty(true_next_item_id))
    print("Recomendação:   ", pretty(best_id))
    print()

    # evidência Markov (transição)
    if c is not None:
        print(f"Explicação: {exp} (Markov count={c})")
    else:
        print(f"Explicação: {exp}")
    print()

    # contexto da sessão
    if (df_events is not None) and (session_id is not None) and (timestamp is not None):
        ctx = get_recent_context(df_events, user_id, session_id, timestamp, n=context_n)
        print("--- Contexto recente (mesma sessão) ---")
        for i, iid in enumerate(ctx, start=1):
            print(f"{i:02d}. {pretty(iid)}")
        print()

    # ranking global do recomendado
    gr = get_global_rank(best_id)
    print("--- Ranking global da música recomendada ---")
    if gr is None or (isinstance(gr, float) and np.isnan(gr)):
        print(f"{pretty(best_id)} não aparece no ranking global calculado.")
    else:
        print(f"{pretty(best_id)} está no ranking global: posição {int(gr)}.")
    print()

    # Top-N do Markov por frequência (se tiver trans_counts)
    print(f"--- Top-{topn_show} próximas músicas possíveis (Markov, por frequência no treino) ---")
    # ordenar candidatos por count (se não tiver count, mantém ordem)
    if len(trans_counts) > 0:
        cand_with_counts = []
        for cid in candidates:
            cc = get_transition_count(current_item_id, cid)
            cand_with_counts.append((cid, -1 if cc is None else cc))
        cand_with_counts.sort(key=lambda x: x[1], reverse=True)
        show = cand_with_counts[:topn_show]
        for i, (cid, cc) in enumerate(show, start=1):
            if cc == -1:
                print(f"{i:02d}. {pretty(cid)}")
            else:
                print(f"{i:02d}. {pretty(cid)} (count={cc})")
    else:
        for i, cid in enumerate(candidates[:topn_show], start=1):
            print(f"{i:02d}. {pretty(cid)}")

    print()
    # Top-N do híbrido (rerank por ALS)
    print(f"--- Top-{min(topn_show, len(scored))} candidatos rerankeados pelo ALS (afinidade) ---")
    scored_sorted = sorted(scored, key=lambda x: x[1], reverse=True)[:topn_show]
    for i, (cid, sc) in enumerate(scored_sorted, start=1):
        print(f"{i:02d}. {pretty(cid)} (als_score={sc:.4f})")

    return {
        "user_id": user_id,
        "current_item_id": current_item_id,
        "recommended_item_id": best_id,
        "best_score": best_score,
        "candidates": candidates,
        "candidate_scores": scored,
        "explanation": exp
    }


In [54]:
import pandas as pd

test_pairs = pd.read_parquet("data/test_pairs_ids.parquet")

print(test_pairs.columns.tolist())
print("Total de linhas:", len(test_pairs))

['user_id', 'item_id', 'next_item_id', 'session_id', 'timestamp']
Total de linhas: 3163359


In [55]:
def hybrid_recommend(user_id: str, current_item_id: str, K: int = 20):
    candidates = markov_topk.get(current_item_id, [])
    candidates = [str(x) for x in candidates][:K]
    candidates = [cid for cid in candidates if cid != str(current_item_id)]

    if len(candidates) == 0:
        return None

    scored = [(cid, als_score(user_id, cid)) for cid in candidates]
    best_id, best_score = max(scored, key=lambda x: x[1])

    explanation = (
        f"Você está ouvindo {pretty(best_id)} porque, entre as músicas mais escutadas depois de {pretty(current_item_id)}, "
        f"é a faixa com mais afinidade com o seu gosto musical."
    )

    return {
        "user_id": user_id,
        "current_item_id": current_item_id,
        "recommended_item_id": best_id,
        "candidates": candidates,
        "candidate_scores": scored,
        "explanation": explanation,
        "best_score": best_score
    }


In [56]:
# TESTE INTERMEDIÁRIO
import textwrap

row = test_pairs.sample(1, random_state=42).iloc[0]
u = row["user_id"]
y = row["item_id"]
true_next = row["next_item_id"]

out = hybrid_recommend(u, y, K=20)

explanation = (
    f"Você está ouvindo {pretty(best_id)} porque, entre as músicas mais escutadas depois de {pretty(current_item_id)}, "
    f"é a faixa com mais afinidade com o seu gosto musical."
)

print("Usuário:", u)
print("Tocando agora (Y):", pretty(y))
print("Recomendado (X):", pretty(out["recommended_item_id"]) if out else None)
print("Próxima real:", pretty(true_next))
print()
print("Explicação:")
print(textwrap.fill(explanation, width=90))


Usuário: user_000899
Tocando agora (Y): Tool — Right In Two
Recomendado (X): Tool — Wings For Marie, Part 1
Próxima real: Tool — Viginti Tres

Explicação:
Você está ouvindo Elliott Smith — Between The Bars porque, entre as músicas mais escutadas
depois de Amon Amarth — Where Death Seems To Dwell, é a faixa com mais afinidade com o seu
gosto musical.


In [57]:
# 0) garanta que test_pairs existe
# import pandas as pd

# if "test_pairs" not in globals():
#     test_pairs = pd.read_parquet("data/test_pairs_ids.parquet")

# # 1) escolha 1 linha aleatória
# row = test_pairs.sample(1, random_state=42).iloc[0]

# # 2) df_context: se não existir, desliga contexto automaticamente
# df_ctx = globals().get("df_context", None)

# # 3) chamar relatório
# out = hybrid_recommend_with_report(
#     user_id=row["user_id"],
#     current_item_id=row["item_id"],
#     session_id=row["session_id"],
#     timestamp=row["timestamp"],
#     K=20,
#     topn_show=10,
#     context_n=5,
#     df_events=df_ctx,   # None = sem contexto
#     true_next_item_id=row["next_item_id"],
# )


In [58]:
# PARAMETRIZAÇÃO DE SCORES

import numpy as np

def minmax_norm(values, eps=1e-12):
    v = np.asarray(values, dtype=float)

    # se tudo é NaN, devolve tudo 0
    if np.all(np.isnan(v)):
        return np.zeros_like(v)

    vmin = np.nanmin(v)
    vmax = np.nanmax(v)

    # se vmin/vmax der NaN ou não há variação, devolve zeros
    if np.isnan(vmin) or np.isnan(vmax) or abs(vmax - vmin) < eps:
        return np.zeros_like(v)

    out = (v - vmin) / (vmax - vmin + eps)
    out = np.where(np.isnan(out), 0.0, out)  # NaN vira 0
    return out

def markov_raw_score(current_item_id: str, cand_item_id: str) -> float:
    c = get_transition_count(str(current_item_id), str(cand_item_id))
    return float(c) if c is not None else 0.0


In [59]:
top_pop = pd.read_parquet("data/global_rank_ids_top200.parquet")
top_pop["item_id"] = top_pop["item_id"].astype(str)

popular_items = top_pop.sort_values("rank")["item_id"].tolist()  # lista em ordem (mais popular primeiro)
popular_set = set(popular_items)


In [60]:
def als_item_sim(item_a: str, item_b: str) -> float:
    ia = item_id_to_index.get(str(item_a))
    ib = item_id_to_index.get(str(item_b))
    if ia is None or ib is None:
        return float("NaN")
    return float(item_factors[ia] @ item_factors[ib])


In [79]:
# FUNÇÃO DE RECOMENDAÇÃO PARA O FALLBACK

def fallback_popular_recommend(user_id: str, current_item_id: str, beta: float = 0.5, N_pop: int = 200):
    user_id = str(user_id)
    current_item_id = str(current_item_id)

    # candidatos populares (em ordem de popularidade)
    candidates = popular_items[:N_pop]
    candidates = [cid for cid in popular_items[:N_pop] if cid != str(current_item_id)]
    if len(candidates) == 0:
        return None

    # scores brutos
    als_raw = [als_score(user_id, cid) for cid in candidates]

    # similaridade com a música atual (se a atual existir no ALS)
    sim_raw = [als_item_sim(current_item_id, cid) for cid in candidates]

    # normalização por min-max dentro do conjunto popular
    als_norm = minmax_norm(als_raw)
    sim_norm = minmax_norm(sim_raw)

    final = beta * als_norm + (1 - beta) * sim_norm
    best_idx = int(np.argmax(final))
    best_id = candidates[best_idx]

    explanation_fb = (
        f"Você está ouvindo {pretty(best_id)} porque, entre as músicas mais populares, "
        f"ela é a melhor combinação entre afinidade com o seu gosto e semelhança com {pretty(current_item_id)}."
    )

    return {
        "recommended_item_id": best_id,
        "candidates": candidates,
        "als_raw": als_raw,
        "sim_raw": sim_raw,
        "als_norm": als_norm,
        "sim_norm": sim_norm,
        "final_scores": final,
        "beta": beta,
        "explanation": explanation_fb,
    }


In [64]:
# FUNÇÃO PRINCIPAL DE RECOMENDAÇÃO EXPLICÁVEL COM MODELO HÍBRIDO

import textwrap

def hybrid_recommend_with_report(
    user_id: str,
    current_item_id: str,
    session_id: str | None = None,
    timestamp=None,
    K: int = 20,
    topn_show: int = 10,
    context_n: int = 5,
    df_events=None,
    true_next_item_id: str | None = None,
    alpha: float = 0.7
):
    user_id = str(user_id)
    current_item_id = str(current_item_id)

    candidates = markov_topk.get(current_item_id, [])
    candidates = [str(x) for x in candidates][:K]

    if not candidates:
        fb = fallback_popular_recommend(user_id, current_item_id, beta=0.6, N_pop=200)

        # garante best_id
        if isinstance(fb, dict):
            best_id = fb.get("recommended_item_id") or fb.get("best_id")
        else:
            best_id = str(fb)

        explanation_fb = (
        f"Você está ouvindo {pretty(best_id)} porque, entre as músicas mais populares, "
        f"ela é a melhor combinação entre afinidade com o seu gosto e semelhança com {pretty(current_item_id)}."
        )

        print("⚠️ Fallback ativado: sem candidatos do Markov para a música atual.")
        print("Música atual:   ", pretty(current_item_id))
        if true_next_item_id is not None:
            print("Próxima real:   ", pretty(true_next_item_id))
        print("Recomendação:   ", pretty(fb["recommended_item_id"]))
        print()
        print("Explicação:")
       
        print(textwrap.fill(explanation_fb, width=90))
        print()
        
        return {
            "user_id": user_id,
            "current_item_id": current_item_id,
            "recommended_item_id": best_id,
            "alpha": alpha,
            "candidates": [],
            "strategy": "fallback_popular",
            "fallback_raw": fb,
        }

    # 1) scores brutos
    markov_raw = [markov_raw_score(current_item_id, cid) for cid in candidates]
    als_raw    = [als_score(user_id, cid) for cid in candidates]

    # 2) normalização
    markov_norm = minmax_norm(markov_raw)
    als_norm    = minmax_norm(als_raw)

    # 3) score final ponderado
    final_scores = alpha * markov_norm + (1 - alpha) * als_norm

    best_idx = int(np.argmax(final_scores))
    best_id = candidates[best_idx]

    if best_id == str(current_item_id):
        # remove e pega o próximo melhor
        ranked = np.argsort(final_scores)[::-1]
        for j in ranked:
            if candidates[j] != str(current_item_id):
                best_id = candidates[j]
                best_idx = int(j)
                break

    # 4) scores ALS para comparação (se houver próxima real)
    als_real = None
    als_rec = als_score(user_id, best_id)

    if true_next_item_id is not None:
        als_real = als_score(user_id, true_next_item_id)

    explanation_norm = (
        f"Você está ouvindo {pretty(best_id)} porque, entre as músicas mais escutadas "
        f"depois de {pretty(current_item_id)}, ela apresentou a melhor combinação entre "
        f"probabilidade de transição e afinidade com o seu gosto musical."
    )

    # ---------------- PRINT RELATÓRIO ----------------

    print("Música atual:   ", pretty(current_item_id))
    if true_next_item_id is not None:
        print("Próxima real:   ", pretty(true_next_item_id))
        print(f"  └ ALS raw score = {als_real:.4f}")
    print("Recomendação:   ", pretty(best_id))
    print(f"  └ ALS raw score  = {als_rec:.4f}")
    print()

    print("Explicação:")
    print(textwrap.fill(explanation_norm, width=90))
    print()

    # comparação direta de afinidade (ALS)
    if als_real is not None:
        if als_rec > als_real:
            print("➡️ A recomendação possui MAIOR afinidade com o gosto do usuário do que a próxima música real.")
        elif als_rec < als_real:
            print("➡️ A próxima música real tinha MAIOR afinidade com o gosto do usuário.")
        else:
            print("➡️ Ambas possuem afinidade equivalente segundo o ALS.")
        print()

    # -------- Contexto da sessão --------
    if df_events is not None and session_id is not None and timestamp is not None:
        ctx = get_recent_context(df_events, user_id, session_id, timestamp, n=context_n)
        if len(ctx) > 0:
            print(f"--- Contexto recente (até {len(ctx)} anteriores na mesma sessão) ---")
            for i, iid in enumerate(ctx, start=1):
                print(f"{i:02d}. {pretty(iid)}")
            print()

    # -------- Top-N Markov (frequência) --------
    print(f"--- Top-{topn_show} próximas músicas possíveis (Markov) ---")
    order_mk = np.argsort(markov_raw)[::-1][:topn_show]
    for rank, idx in enumerate(order_mk, start=1):
        cid = candidates[idx]
        print(f"{rank:02d}. {pretty(cid)} (count={markov_raw[idx]})")
    print()

    # -------- Posição relativa da música real no ranking ALS dos candidatos --------
    if true_next_item_id is not None:
        x_real = str(true_next_item_id)
        if x_real in candidates:
            idx_real = candidates.index(x_real)
            als_real_in_candidates = als_raw[idx_real]
            sorted_scores = sorted(als_raw, reverse=True)
            rank_real = sorted_scores.index(als_real_in_candidates) + 1
            print("--- Posição da próxima música real entre os candidatos ---")
            print("A música real está entre os candidatos do Markov.")
            print(f"Posição no ranking ALS dos candidatos: {rank_real}/{len(candidates)}")
        else:
            print("--- Posição da próxima música real entre os candidatos ---")
            print("A música real NÃO estava entre os candidatos do Markov (fora do Top-K).")
        print()

    # -------- Top-N final (score ponderado) --------
    print(f"--- Top-{topn_show} candidatos após reranking híbrido ---")
    order_final = np.argsort(final_scores)[::-1][:topn_show]
    for rank, idx in enumerate(order_final, start=1):
        cid = candidates[idx]
        print(
            f"{rank:02d}. {pretty(cid)} | "
            f"markov={markov_norm[idx]:.3f} | "
            f"als={als_norm[idx]:.3f} | "
            f"score_final={final_scores[idx]:.3f}"
        )

    return {
        "user_id": user_id,
        "current_item_id": current_item_id,
        "recommended_item_id": best_id,
        "alpha": alpha,
        "candidates": candidates,
        "markov_raw": markov_raw,
        "als_raw": als_raw,
        "markov_norm": markov_norm,
        "als_norm": als_norm,
        "final_scores": final_scores,
    }


In [76]:
# TESTE FINAL

import pandas as pd
import numpy as np

# 1) escolher uma sessão aleatória (sem seed)
sess_id = test_pairs["session_id"].sample(1).iloc[0]

df_sess = test_pairs[test_pairs["session_id"] == sess_id].copy()

# ordenar por tempo para respeitar a sequência
df_sess["timestamp"] = pd.to_datetime(df_sess["timestamp"], utc=True, errors="coerce")
df_sess = df_sess.sort_values("timestamp")

# segurança: sessão precisa ter pelo menos 2 músicas
if len(df_sess) < 2:
    raise ValueError("Sessão sorteada tem menos de 2 músicas. Rode a célula novamente.")

# 2) escolher uma música aleatória da sessão (não a última)
idx = np.random.randint(0, len(df_sess) - 1)
row = df_sess.iloc[idx]

print("Usuário:", row["user_id"])
print("Sessão escolhida:", sess_id)
print("Posição na sessão:", idx, "/", len(df_sess) - 1)
print()

# 3) chamar o híbrido
out = hybrid_recommend_with_report(
    user_id=row["user_id"],
    current_item_id=row["item_id"],     # música tocando agora
    session_id=row["session_id"],
    timestamp=row["timestamp"],
    K=20,
    topn_show=10,
    context_n=5,
    df_events=df_context,               # ou None
    true_next_item_id=row["next_item_id"],
    alpha=0.7
)


Usuário: user_000667
Sessão escolhida: 583
Posição na sessão: 269 / 1089

⚠️ Fallback ativado: sem candidatos do Markov para a música atual.
Música atual:    Michael Armstrong — Don'T Panic
Próxima real:    Michael Armstrong — Trouble
Recomendação:    James Blunt — Goodbye My Lover

Explicação:
Você está ouvindo James Blunt — Goodbye My Lover porque, entre as músicas mais populares,
ela é a melhor combinação entre afinidade com o seu gosto e semelhança com Michael
Armstrong — Don'T Panic.



In [80]:
# TESTE FINAL COM FALLBACK

import numpy as np
import pandas as pd
import textwrap

# ---------------- TESTE DE FALLBACK (RELATÓRIO) ----------------

# 1) escolher sessão + música aleatória
row = test_pairs.sample(1).iloc[0]

user_id = row["user_id"]
current_item_id = row["item_id"]
true_next_item_id = row.get("next_item_id", None)
session_id = row.get("session_id", None)
timestamp = row.get("timestamp", None)

# 2) rodar fallback (popular + ALS + similaridade)
out_fb = fallback_popular_recommend(
    user_id=user_id,
    current_item_id=current_item_id,
    beta=0.6,
    N_pop=200
)

best_id = out_fb["recommended_item_id"]

# 3) scores ALS para comparação
als_real = als_score(user_id, true_next_item_id) if true_next_item_id is not None else None
als_rec = als_score(user_id, best_id)

# ---------------- PRINT RELATÓRIO ----------------

print("⚠️ Fallback ativado: modelo de Markov sem candidatos válidos.")
print()

print("Música atual:   ", pretty(current_item_id))
if true_next_item_id is not None:
    print("Próxima real:   ", pretty(true_next_item_id))
    print(f"  └ ALS(u, X_real) = {als_real:.4f}")
print("Recomendação:   ", pretty(best_id))
print(f"  └ ALS(u, X_rec)  = {als_rec:.4f}")
print()

print("Explicação:")
print(textwrap.fill(out_fb["explanation"], width=90))
print()

# comparação direta ALS
if als_real is not None:
    if als_rec > als_real:
        print("➡️ A recomendação possui MAIOR afinidade com o gosto do usuário do que a próxima música real.")
    elif als_rec < als_real:
        print("➡️ A próxima música real tinha MAIOR afinidade com o gosto do usuário.")
    else:
        print("➡️ Ambas possuem afinidade equivalente segundo o ALS.")
    print()

# -------- Contexto da sessão --------
if df_context is not None and session_id is not None and timestamp is not None:
    ctx = get_recent_context(df_context, user_id, session_id, timestamp, n=5)
    if len(ctx) > 0:
        print(f"--- Contexto recente (até {len(ctx)} anteriores na mesma sessão) ---")
        for i, iid in enumerate(ctx, start=1):
            print(f"{i:02d}. {pretty(iid)}")
        print()

# -------- Top-N populares (fallback) --------
print("--- Top-10 candidatos populares após reranking de fallback ---")
order_fb = np.argsort(out_fb["final_scores"])[::-1][:10]

for rank, idx in enumerate(order_fb, start=1):
    cid = out_fb["candidates"][idx]
    print(
        f"{rank:02d}. {pretty(cid)} | "
        f"als={out_fb['als_norm'][idx]:.3f} | "
        f"sim={out_fb['sim_norm'][idx]:.3f} | "
        f"score_final={out_fb['final_scores'][idx]:.3f}"
    )


⚠️ Fallback ativado: modelo de Markov sem candidatos válidos.

Música atual:    My Chemical Romance — Dead!
Próxima real:    My Chemical Romance — Cemetary Drive
  └ ALS(u, X_real) = 0.9091
Recomendação:    Panic At The Disco — Lying Is The Most Fun A Girl Can Have Without Taking Her Clothes Off
  └ ALS(u, X_rec)  = 1.0089

Explicação:
Você está ouvindo Panic At The Disco — Lying Is The Most Fun A Girl Can Have Without
Taking Her Clothes Off porque, entre as músicas mais populares, ela é a melhor combinação
entre afinidade com o seu gosto e semelhança com My Chemical Romance — Dead!.

➡️ A recomendação possui MAIOR afinidade com o gosto do usuário do que a próxima música real.

--- Contexto recente (até 5 anteriores na mesma sessão) ---
01. The White Stripes — I Can Learn
02. Puddle Of Mudd — Think
03. Wolfgang Amadeus Mozart — Horn Concerto No. 4 In E-Flat, Kv 495: I. Allegro Moderato
04. Skillet — Kill Me, Heal Me
05. Death From Above 1979 — Losing Friends

--- Top-10 candidatos popu

In [67]:
# AVALIAÇÃO DO MODELO HÍBRIDO
# Hit@K e MRR

import numpy as np
from tqdm import tqdm

def hybrid_rank_list(user_id, current_item_id, K=20, alpha=0.7):
    candidates = markov_topk.get(str(current_item_id), [])
    candidates = [str(x) for x in candidates][:K]

    if not candidates:
        return []

    markov_raw = [markov_raw_score(current_item_id, cid) for cid in candidates]
    als_raw    = [als_score(user_id, cid) for cid in candidates]

    markov_norm = minmax_norm(markov_raw)
    als_norm    = minmax_norm(als_raw)

    final_scores = alpha * markov_norm + (1 - alpha) * als_norm

    order = np.argsort(final_scores)[::-1]

    ranked_items = [candidates[i] for i in order]
    return ranked_items


def evaluate_hybrid(pairs_df, K=10, alpha=0.7):
    hits = []
    reciprocal_ranks = []

    for _, row in tqdm(pairs_df.iterrows(), total=len(pairs_df)):
        user_id = str(row["user_id"])
        current_item = str(row["item_id"])
        true_next = str(row["next_item_id"])

        ranked = hybrid_rank_list(user_id, current_item, K=K, alpha=alpha)

        if true_next in ranked:
            rank = ranked.index(true_next) + 1
            hits.append(1)
            reciprocal_ranks.append(1 / rank)
        else:
            hits.append(0)
            reciprocal_ranks.append(0)

    hit_k = np.mean(hits)
    mrr = np.mean(reciprocal_ranks)

    return hit_k, mrr


In [69]:
# RODAR A AVALIAÇÃO

K = 10
alpha = 0.7

hit10, mrr10 = evaluate_hybrid(pairs_valid, K=K, alpha=alpha)

print(f"Hybrid Hit@{K}: {hit10:.4f}")
print(f"Hybrid MRR: {mrr10:.4f}")


NameError: name 'pairs_valid' is not defined